In [6]:
import datetime as dt
import pandas as pd
import numpy as np

import matplotlib as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [ ]:
df = pd.read_excel('../Dataset/Online Retail.xlsx')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df[['InvoiceNo','StockCode']].duplicated().sum() - df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
df[df[['CustomerID','InvoiceNo','StockCode']].duplicated()==True]

In [ ]:
df[((df['CustomerID']==17908.0)&(df['InvoiceNo']==536409)&(df['StockCode']=='90199C'))]

In [ ]:
df.Quantity.describe()

In [ ]:
df[df['Quantity']<0].shape[0]

In [ ]:
df[df['Quantity']<0].InvoiceNo.str.startswith('C').value_counts(dropna=False)

In [ ]:
df[df['Quantity']<0][df[df['Quantity']<0].InvoiceNo.str.startswith('C').isnull()].Description.value_counts()

In [ ]:
df.boxplot('Quantity',showfliers=False)

In [ ]:
df['OrderCancel'] = df['InvoiceNo'].str.startswith('C')
df[df['Quantity']<0]['OrderCancel'].value_counts(dropna=False)

In [ ]:
df.isnull().sum()

BASIC DATA CLEANING

In [ ]:
# df = df.drop_duplicates()
df = df.drop_duplicates(subset = ['InvoiceNo','StockCode','Quantity'])
df = df[df['CustomerID'].notnull()]
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df[df['Quantity'] > 0]
df.shape

# Time Series Trends Analysis

In [ ]:
df['InvoiceDate'].min()

In [ ]:
df['InvoiceDate'].max()

In [ ]:
df.head(1)

In [ ]:
df['order_year_month'] = df['InvoiceDate'].dt.strftime('%Y-%m')

In [ ]:
df.groupby('order_year_month').InvoiceNo.nunique().plot()

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(montly_orders_df)

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(df.groupby('order_year_month').InvoiceNo.nunique())

In [ ]:
ax = df.groupby('order_year_month').InvoiceNo.nunique().plot(figsize=(10,7),grid=True)
ax.set_title('Total number of order overtime')
ax.set_ylabel('number of orders')

In [ ]:
df[df['order_year_month']=='2011-12']['InvoiceDate'].min(),df[df['order_year_month']=='2011-12']['InvoiceDate'].max()

In [ ]:
###Filter out incomplete data for December 2011
df = df[df['order_year_month']<'2011-12']

In [ ]:
df.groupby('order_year_month').InvoiceNo.nunique().plot(grid=True,figsize=(10,7))

In [ ]:
sale_df = df.groupby('order_year_month').InvoiceNo.nunique().reset_index()
sale_df.columns = ['OrderDate','Quantity']

In [ ]:
revenu_df = df.groupby('order_year_month').Revenue.sum().reset_index()
revenu_df.columns = ['OrderDate','Revenue']

In [ ]:
sale_df = sale_df.merge(revenu_df,on='OrderDate',how='left')
sale_df

In [ ]:
sale_df.to_csv('sales.csv',index=False)

Visualize Montly Revenue Data

In [ ]:
df['Revenue'] = df['Quantity']*df['UnitPrice']
ax = df.groupby('order_year_month').Revenue.sum().plot(figsize=(10,7),grid=True)
ax.set_title('Total Revenue Over Time')
ax.set_ylabel('Sale Revenue')
ax.set_xlabel('Date')

# Repeat Purchases Analysis 

In [ ]:
####Calculate montly repeat purchases
grouped_df = df.groupby(['order_year_month','CustomerID'])['InvoiceNo'].nunique().reset_index()
repeat_customer_df = grouped_df[grouped_df['InvoiceNo']>=2].groupby('order_year_month').CustomerID.count().reset_index()
repeat_customer_df.columns=['orderDate','nbr_repeat_customers']

In [ ]:
repeat_customer_df

In [ ]:
full_customer_df = grouped_df.groupby('order_year_month')\
.CustomerID.count().reset_index()
full_customer_df.columns = ['orderDate','nbr_customer']

In [ ]:
full_customer_df.columns

In [ ]:
repeat_customer_df = repeat_customer_df.merge(full_customer_df,on='orderDate',how='left')

In [ ]:
repeat_customer_df

In [ ]:
repeat_customer_df['repeated_cst_pct'] = round(100*(repeat_customer_df['nbr_repeat_customers']\
                                                    /repeat_customer_df['nbr_customer']),2)

In [ ]:
repeat_customer_df

In [ ]:
grouped_df = df.groupby(['order_year_month','CustomerID'])\
.agg({'InvoiceNo':'nunique','Revenue':'sum'}).reset_index()

In [ ]:
grouped_df

In [ ]:
repeat_revenue_df = grouped_df[grouped_df['InvoiceNo']>=2]\
.groupby('order_year_month').Revenue.sum().reset_index()
repeat_revenue_df.columns = ['orderDate','repeat_customer_revenue']
full_revenue_df =  grouped_df.groupby('order_year_month')\
.Revenue.sum().reset_index()
full_revenue_df.columns=['orderDate','revenue']
repeat_revenue_df = repeat_revenue_df.merge(full_revenue_df,on='orderDate',how='left')
repeat_revenue_df['repeat_customer_revenue_contribution'] = round(100*(repeat_revenue_df['repeat_customer_revenue']\
                                                                 /repeat_revenue_df['revenue']),2)

In [ ]:
repeat_revenue_df

In [ ]:
repeat_customer_df

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
repeat_customer_df[['orderDate','repeated_cst_pct']].set_index('orderDate')\
.plot(kind='line',ax=ax,linewidth=3,color='grey')
repeat_revenue_df[['orderDate','repeat_customer_revenue_contribution']].set_index('orderDate')\
.plot(kind='line',ax=ax,color='grey',linewidth = 3)
ax.get_legend().remove()
xdata2 = lines[0].get_xdata()[1:5]
ydata2 = lines[0].get_ydata()[1:5]
ax.plot(xdata2,ydata2,color='blue',linewidth=3)
xdata2 = lines[0].get_xdata()[[1,2,3,4]]
ydata2 = lines[0].get_ydata()[[1,2,3,4]]
ax.plot(xdata2,ydata2,color='blue',linewidth=3)
xdata2 = lines[0].get_xdata()[[8,9,10]]
ydata2 = lines[0].get_ydata()[[8,9,10]]
ax.plot(xdata2,ydata2,color='blue',linewidth=3)
xdata2 = lines[1].get_xdata()[[1,2,3,4]]
ydata2 = lines[1].get_ydata()[[1,2,3,4]]
ax.plot(xdata2,ydata2,color='orange',linewidth=3)
xdata2 = lines[1].get_xdata()[[8,9,10]]
ydata2 = lines[1].get_ydata()[[8,9,10]]
ax.plot(xdata2,ydata2,color='orange',linewidth=3)
ax.spines[['top','right']].set_visible(False)
lines = ax.lines
# xdata = lines[0].get_xdata()[3]
# ydata = lines[0].get_ydata()[3]
# ax.text(xdata,ydata+1,ydata)
# xdata = lines[1].get_xdata()[3]
# ydata = lines[1].get_ydata()[3]
# ax.text(xdata,ydata+1,ydata)
# xdata = lines[0].get_xdata()[8]
# ydata = lines[0].get_ydata()[8]
# ax.text(xdata,ydata+1,ydata)
# xdata = lines[1].get_xdata()[8]
# ydata = lines[1].get_ydata()[8]
# ax.text(xdata,ydata+1,ydata)
xdata = lines[0].get_xdata()[2]
ydata = lines[0].get_ydata()[2]
ax.text(xdata,ydata+2,'avg = 20%')
xdata = lines[0].get_xdata()[8]
ydata = lines[0].get_ydata()[8]
ax.text(xdata,ydata+2,'avg = 22%')
xdata = lines[1].get_xdata()[2]
ydata = lines[1].get_ydata()[2]
ax.text(xdata,ydata+3,'avg = 42%')
xdata = lines[1].get_xdata()[8]
ydata = lines[1].get_ydata()[8]
ax.text(xdata,ydata+2,'avg = 52%',rotation=20)
# for i in range(2):
#     xdata = lines[i].get_xdata()
#     ydata = lines[i].get_ydata()
#     for x, y in zip(xdata,ydata):
#         ax.text(x,y+1,y)

Insight: Tỷ lệ % repeat customers giai đoạn 4 tháng đầu năm và 3 tháng 8, 9, 10 dao động 20%, 
có xu hướng tăng mạnh vào các tháng cuối năm. Tuy nhiên ở giai đoạn đầu năm 20% repeat customers chỉ mang lại tối đa 44.83% doanh thu,
ngược lại ở giai đoạn sau mặc dù tỷ lệ khách hàng trung thành cũng chỉ chiếm có 22% nhưng doanh thu mang lai lên tới 52%.
Như vậy 1% repeat customer tăng thêm trong giai đoạn này giúp tăng thêm 5% doanh thu.
ACTION: đưa ra các gói kích thích khách hàng cũ mua hàng trong giai đoạn này.

# Trending items over time

In [ ]:
data_item_df = df.groupby(['order_year_month','StockCode']).Quantity.sum().reset_index()

In [ ]:
# Rank items by the last month sales
data_item_df[data_item_df['order_year_month']=='2011-11'].sort_values(by='Quantity',ascending=False)

In [ ]:
best_seller_items = [23084,84826,22197,22086,'85099B']
trending_items_df = df[df['StockCode'].isin(best_seller_items)]\
.groupby(['order_year_month','StockCode']).Quantity.sum()

In [ ]:
ax = trending_items_df.unstack().fillna(0).plot()
ax.set_ylabel('number of orders')
ax.set_xlabel('date')
ax.set_title('Item Trends over Time')

In [ ]:
ax = trending_items_df.unstack().fillna(0).plot(color=['dimgray','grey','darkgray','Blue','silver'],figsize=[10,5],linewidth=3)
ax.set_ylabel('number of orders')
ax.set_xlabel('date')
ax.set_title('Item Trends over Time')

In [ ]:
trending_items_Revenue_df = df[df['StockCode'].isin(best_seller_items)].groupby(['order_year_month','StockCode']).Revenue.sum()
ax = trending_items_Revenue_df.unstack().fillna(0).plot(color=['dimgray','grey','darkgray','Blue','silver'],figsize=[10,5],linewidth=3)
ax.set_ylabel('total revenue')
ax.set_xlabel('date')
ax.set_title('Item Revenue per month')

In [ ]:
df[((df['StockCode']==84826)&(df['order_year_month']=='2011-11'))]

In [ ]:
df['dayinweek'] = df['InvoiceDate'].dt.weekday

In [ ]:
df['dayinweek'].value_counts()

In [ ]:
#### san pham nao nen ban vao thoi gian nao?Phai loai bo cac san pham co discount hoac trong chien dich campaign.
fig,ax = plt.subplots(5,1,figsize=(10,8))
i=0
colors =['blue','orange','red','grey','green']
for ax1 in ax:
    df[df['StockCode']==best_seller_items[i]].groupby('dayinweek')['Quantity'].sum()\
    .plot(kind='line',ax=ax1,color=colors[i])
    ax1.spines[['bottom','top','left','right']].set_color('#DCDCDC')
    ticks = ax1.get_xticks()
    ax1.set_xlabel('')
#     ax1.set_xticklabels(['Monday','Tuesday','Wenesday','Thursday','Friday','Saturday','Sunday'])
    ax1.set_ylabel(best_seller_items[i])
    i+=1
ax1.set_xlabel('Month Of Order Date')
plt.xticks(range(7),['Monday','Tuesday','Wenesday','Thursday','Friday','Saturday','Sunday'])

In [ ]:
df['day'] = df['InvoiceDate'].dt.day
df['day'].value_counts()

In [ ]:
fig,ax = plt.subplots(5,1,figsize=(10,8))
i=0
colors =['blue','orange','red','grey','green']
for ax1 in ax:
    df[df['StockCode']==best_seller_items[i]].groupby('day')['Quantity'].sum()\
    .plot(kind='line',ax=ax1,color=colors[i])
    ax1.spines[['bottom','top','left','right']].set_color('#DCDCDC')
    ticks = ax1.get_xticks()
    ax1.set_xlabel('')
#     ax1.set_xticklabels(['Monday','Tuesday','Wenesday','Thursday','Friday','Saturday','Sunday'])
    ax1.set_ylabel(best_seller_items[i])
    i+=1